In [ ]:
! pip install python-dotenv langchain langchain-community langchainhub openai tiktoken azure-ai-documentintelligence azure-identity azure-search-documents==11.4.0b8
! pip install langchain --upgrade

In [38]:
# Global Variables
from azure.search.documents.indexes.models import (
    ScoringProfile,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    TextWeights)

# Define fileds for the index of the corpus

fields = [
SimpleField(
    name="id",
    type=SearchFieldDataType.String,
    key=True,
    filterable=True,
),
SearchableField(
    name="content",
    type=SearchFieldDataType.String,
    searchable=True,
),
SearchField(
    name="content_vector",
    type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
    searchable=True,
    vector_search_dimensions=1536,
    vector_search_configuration="default",
),
SearchableField(
    name="metadata",
    type=SearchFieldDataType.String,
    searchable=True,
),
# Additional field to store which engine got an answer
SearchableField(
    name="engine",
    type=SearchFieldDataType.String,
    searchable=True,
),
# Additional field for filtering on document source
SimpleField(
    name="source",
    type=SearchFieldDataType.String,
    filterable=True,
),
]


In [40]:
# Set environment variables
# Called multiple times in differenct cells, you can run them independently
from dotenv import load_dotenv
import os

def set_env():
    load_dotenv()  # take environment variables from .env.
    os.environ["openai.api_type"] = os.getenv("openai.api_type")
    os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
    os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
    global doc_intelligence_endpoint
    doc_intelligence_endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")

    global doc_intelligence_key
    doc_intelligence_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")

    global api_version
    api_version = os.getenv("openai.api_version")

    global ada_deployed_model 
    ada_deployed_model = os.getenv("ada")

    global gpt4_deployed_model
    gpt4_deployed_model = os.getenv("gpt4")
    
    global vector_store_address
    vector_store_address = os.getenv("AZURE_SEARCH_ENDPOINT")

    global vector_store_password
    vector_store_password = os.getenv("AZURE_SEARCH_ADMIN_KEY")

set_env()

In [ ]:
# Defines the function to chunk and embed a document

from langchain import hub
from langchain.chat_models import AzureChatOpenAI
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores.azuresearch import AzureSearch

from dotenv import load_dotenv
import os

def process_file(file_path, index_name):
    # Set environment variables
    set_env()

    # Initiate Azure AI Document Intelligence to load the document. You can either specify file_path or url_path to load the document.
    loader = AzureAIDocumentIntelligenceLoader(
        file_path=file_path,
        api_key=doc_intelligence_key,
        api_endpoint=doc_intelligence_endpoint,
        api_model="prebuilt-layout",
    )
    docs = loader.load()

    # Split the document into chunks base on markdown headers.
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]
    text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

    docs_string = docs[0].page_content
    splits = text_splitter.split_text(docs_string)

    print("Length of splits: " + str(len(splits)))

    # Embed the splitted documents and insert into Azure Search vector store
    # openai.api_base = os.getenv("openai.api_base")
    import openai

    openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")

    aoai_embeddings = AzureOpenAIEmbeddings(
        azure_deployment=ada_deployed_model,
        openai_api_version=api_version,  # e.g., "2023-07-01-preview"
    )

    vector_store: AzureSearch = AzureSearch(
        azure_search_endpoint=vector_store_address,
        azure_search_key=vector_store_password,
        index_name=index_name,
        embedding_function=aoai_embeddings.embed_query,
    )

    vector_store.add_documents(documents=splits)

    

In [34]:
# Let's create two indexes 
# You can comment out this two lines, after the first run.
# After the first indexes will exist in your Azure Search service. No need to create them again.
# Here for practicality, we create two indexes, but in the final implementation, this could be totally different vector databases/stores
process_file("../sample_docs/pdf-sample.pdf", "pdf-sample")
process_file("../sample_docs/Mac.pdf", "mac")


Length of splits: 1
Length of splits: 2


In [ ]:
# Defines the function to store prompts and which index to search
# This define our corpus of knowledge , of which embedding store (vector store or db) to search

def feed_corpus(question, engine):
    # Set environment variables
    set_env()

    import openai

    openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")

    aoai_embeddings = AzureOpenAIEmbeddings(
        azure_deployment=ada_deployed_model,
        openai_api_version=api_version,  # e.g., "2023-07-01-preview"
    )

    vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
    vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")



    vector_store: AzureSearch = AzureSearch(
        azure_search_endpoint=vector_store_address,
        azure_search_key=vector_store_password,
        index_name="corpus",
        embedding_function=aoai_embeddings.embed_query,
        fields=fields
    )

    
    vector_store.add_texts(
        [question],
        [
            { "content": question, "engine": engine}
        ]
        
        )

In [32]:
def ask_corpus(question,  top=3):
    # Set environment variables
    set_env()
    import openai

    openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")

    aoai_embeddings = AzureOpenAIEmbeddings(
        azure_deployment=ada_deployed_model,
        openai_api_version=api_version,  # e.g., "2023-07-01-preview"
    )

    vector_store: AzureSearch = AzureSearch(
        azure_search_endpoint=vector_store_address,
        azure_search_key=vector_store_password,
        index_name="corpus",
        embedding_function=aoai_embeddings.embed_query,
        fields=fields
    )

    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    retrieved_docs = retriever.get_relevant_documents(
        "<your question>"
    )

    if len(retrieved_docs) == 0: # The corpus does not know which DB has the answer
        return 1,"none"
    
    # there could be multiple answers, we just take the first one in this version

    print(retrieved_docs[0].page_content)
    print(retrieved_docs[0].metadata)
    
    engine = retrieved_docs[0].metadata["engine"]
    return 0, engine

    

In [33]:
# Cell for testing the functions
# feed_corpus("Who can read a PDF file?", "pdf-sample")
print(ask_corpus("Who can read a PDF file?",3))

Who can read a PDF file?
{'id': 'MmYwYThkOWUtNTE0OC00MDQyLTlkYWYtZDM1OTg3MDcxN2Nk', 'content': 'Who can read a PDF file?', 'engine': 'pdf-sample'}
(0, 'pdf-sample')


In [44]:
# Cell for setting up the RAG 
def ask_llm_rag(index_name,question):
    found = -1 
    aoai_embeddings = AzureOpenAIEmbeddings(
        azure_deployment=ada_deployed_model,
        openai_api_version=api_version,  # e.g., "2023-07-01-preview"
    )
    # Set environment variables
    vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=aoai_embeddings.embed_query,
    )
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    retrieved_docs = retriever.get_relevant_documents(
        "<your question>"
    )

    if len(retrieved_docs) == 0: # This vector store does not has anwers
       found = 1
    else:
       found = 0

    # Use a prompt for RAG that is checked into the LangChain prompt hub (https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=989ad331-949f-4bac-9694-660074a208a7)
    prompt = hub.pull("rlm/rag-prompt")
    llm = AzureChatOpenAI(
        openai_api_version=api_version,  # e.g., "2023-07-01-preview"
        azure_deployment=gpt4_deployed_model,
        temperature=0,
    )


    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)


    rag_chain_pdf = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )   
    answer = rag_chain_pdf.invoke(question)
    return(found,answer)

In [45]:
# Cell for testing the RAG
found, ans = ask_llm_rag("pdf-sample", "Who can read a PDF file?")
print(ans)
print(found)

Anyone, anywhere can read a PDF file. All that is needed is the free Adobe Acrobat Reader. This software is easy to download and can be freely distributed by anyone.


In [ ]:
# Multiple index (in the future) multile vector stores
# The corpus is the vector store that contains which vector store has the answer

# We will do two iterartions
# First interation our corupus will be empty, we will need to query al vector stores
# Second iteration we will have a corpus and we will know which vector store has the answer

# Fist iteration
vetor_stores = ["pdf-sample", "mac"]
question = "Who can read a PDF file?"
for store in vetor_stores:
    print(ask_llm_rag(store, question))
    